In [1]:
import pandas as pd
import os
import cv2
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch
import torchvision 
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [309]:
X = np.load("C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\test.npy", allow_pickle=True)

In [310]:
lenth = len(X)

#id
id = []
for i in range (lenth):
    for j in range (len(X[i])):
        id.append(i)
df1 = pd.DataFrame({"id":id})

#x_min
x_min = []
for i in range (lenth):
    for j in range (len(X[i])):
        x_min.append(X[i][j][0])
df2 = pd.DataFrame({"x_min":x_min})

#y_min
y_min = []
for i in range (lenth):
    for j in range (len(X[i])):
        y_min.append(X[i][j][1])
df3 = pd.DataFrame({"y_min":y_min})

#x_max
x_max = []
for i in range (lenth):
    for j in range (len(X[i])):
        x_max.append(X[i][j][2])
df4 = pd.DataFrame({"x_max":x_max})

#y_max
y_max = []
for i in range (lenth):
    for j in range (len(X[i])):
        y_max.append(X[i][j][3])
df5 = pd.DataFrame({"y_max":y_max})

#df
df = pd.concat([df1, df2, df3, df4, df5], axis = 1)

In [311]:
class TrainDataset (Dataset):
    
    def __init__(self, annotation_file):
        super().__init__()
        self.train_df = annotation_file
        self.image_ids = self.train_df["id"].unique()
    
    
    def __getitem__(self, index: int):
        
        image_id = self.image_ids[index]
        bboxes = self.train_df[self.train_df["id"]==image_id]
        
        #image
        img = cv2.imread("C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\wider_face_train\\" + str(index) +".jpg", cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img = img/255
        img = torchvision.transforms.ToTensor()(img)
        
        #bboxes
        df_0 = df[df.id==index]
        df_1 = df_0[['x_min', 'y_min', "x_max", "y_max"]]
        bboxes = torchvision.transforms.ToTensor()(df_1.to_numpy()).view(-1, 4)
        
        
        #labels
        labels = torch.ones(bboxes.size()[0], 1, dtype=torch.int64).view(bboxes.size()[0])
        
        #target
        d_boxes = {"boxes": bboxes,
                  "image_id": torch.tensor(index),
                  "labels": labels}
        
        return img, d_boxes
        
        
    def __len__(self):
        return self.image_ids.shape[0]

        
        

In [312]:
train_dataset = TrainDataset(df)

In [313]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_data_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn)

In [29]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [30]:
#загрузка модели
os.chdir("C:\\Users\\alina\\Downloads")
model.load_state_dict(torch.load("model60.pth", map_location='cpu'))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [23]:
#проверка модели (рисовалка)
image = cv2.imread(r"C:\\Users\\alina\jupiter\Wider_face_dataset_2\",cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
image = image/255
image = torchvision.transforms.ToTensor()(image)
out = model ([image])

out_np = out[0]["boxes"]
out_np = out_np.detach().numpy().astype(int)


def segment():
    image = cv2.imread(r"C:\Users\alina\Downloads\\presidents.jpg", cv2.IMREAD_COLOR)
    for j in range(len(out_np)):
        img = cv2.rectangle(image,(out_np[j][0],out_np[j][1]),(out_np[j][2],out_np[j][3]),(0,255,0),1)
        os.chdir("\\Users\\alina\\jupiter\\Wider_face_dataset_2\\df_test\\")
        cv2.imwrite("presidents_test_22.jpg", img)
            
segment()

In [334]:
#алгоритм обучения

In [322]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [222]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = 0.005, momentum = 0.9, weight_decay = 0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)
num_epochs = 1

itr = 1
for epoch in range(num_epochs):
    
    for images, targets in train_data_loader:
        
        try:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
            loss_dict = model(images, targets)
        
            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()
        
            optimizer.zero_grad()
            losses.backward()
            optimizer. step()
        
            if itr % 1 == 0:
                print (f"Iteration #{itr} loss: {loss_value}")
            
            itr+=1
        
            lr_scheduler.step()
        except:
            continue
            
    print (f"Epoch #{epoch} loss: {loss_value}")

Iteration #1 loss: 0.7439466714859009
Iteration #2 loss: 2.032008171081543
Iteration #3 loss: 1.1407525539398193
Iteration #4 loss: 1.0950628519058228
Iteration #5 loss: 0.7656931281089783
Iteration #6 loss: 0.7749078273773193
Iteration #7 loss: 1.1257145404815674
Iteration #8 loss: 0.2540658116340637
Iteration #9 loss: 0.36855995655059814
Iteration #10 loss: 1.489402413368225
Iteration #11 loss: 0.5196669101715088
Iteration #12 loss: 1.4038856029510498
Iteration #13 loss: 0.571124792098999
Iteration #14 loss: 0.520473301410675
Iteration #15 loss: 0.17243817448616028
Iteration #16 loss: 0.2976207733154297
Iteration #17 loss: 0.783954381942749
Iteration #18 loss: 1.490311622619629
Iteration #19 loss: 0.3068472743034363
Iteration #20 loss: 0.2986435890197754
Iteration #21 loss: 0.17318515479564667
Iteration #22 loss: 0.2867525815963745
Iteration #23 loss: 0.3339443802833557
Iteration #24 loss: 1.275449514389038
Iteration #25 loss: 1.0307732820510864
Iteration #26 loss: 0.7875310182571411

AssertionError: All bounding boxes should have positive height and width. Found invalid box [869.57421875, 59.83524703979492, 870.8759765625, 59.83524703979492] for target at index 0.

In [31]:
image = cv2.imread(r"C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\wider_face_train\\0.jpg", cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
image = image/255
image = torchvision.transforms.ToTensor()(image)
out = model ([image])
print (out)
print (out[0])
print (out[0]["boxes"])
print (out[0]["boxes"].detach().numpy())
print(out[0]["scores"].detach().numpy())

[{'boxes': tensor([[7.8035e+02, 1.9266e+02, 8.0401e+02, 2.3358e+02],
        [2.2377e+02, 1.6779e+02, 2.5869e+02, 2.0394e+02],
        [4.3162e+02, 1.9318e+02, 4.6482e+02, 2.3184e+02],
        [1.1186e+01, 2.0150e+02, 4.2048e+01, 2.3706e+02],
        [7.7477e+02, 1.9911e+02, 8.1249e+02, 2.2433e+02],
        [8.2318e+01, 2.0094e+02, 1.3196e+02, 2.6288e+02],
        [4.2289e+02, 1.8625e+02, 4.4867e+02, 2.3176e+02],
        [4.2406e+02, 1.8475e+02, 4.7716e+02, 2.4638e+02],
        [2.0882e+02, 1.5696e+02, 2.5915e+02, 2.1560e+02],
        [7.7846e+02, 1.8461e+02, 8.0028e+02, 2.2115e+02],
        [5.1711e+02, 1.6871e+02, 5.6206e+02, 2.2323e+02],
        [9.2963e+01, 2.1449e+02, 1.2289e+02, 2.4918e+02],
        [5.7621e+02, 1.5937e+02, 6.0874e+02, 1.8948e+02],
        [7.8534e+02, 1.8550e+02, 8.1259e+02, 2.2854e+02],
        [5.2518e+02, 1.7690e+02, 5.5823e+02, 2.1237e+02],
        [9.6782e+01, 2.0681e+02, 1.3060e+02, 2.4069e+02],
        [5.1685e+02, 1.7044e+02, 5.3718e+02, 2.1753e+02],
   

In [ ]:
for i in range(len(out[0]["boxes"].detach().numpy())):
    

In [ ]:
"C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\wider_face_train\\0.jpg"

In [34]:
#проверка модели (рисовалка)
image = cv2.imread(r"C:\Users\alina\Downloads\\presidents.jpg", cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
image = image/255
image = torchvision.transforms.ToTensor()(image)
out = model ([image])


def nms (boxes, conf_threshold=0.7, iou_threshold=0.4):
    bbox_list_thresholded = []
    bbox_list_new = []

    boxes_sorted = sorted (boxes, reverse=True, key = lambda x : x[5])
    for box in boxes_sorted:
        if box[5] > conf_threshold:
            bbox_list_thresholded.append(box)
        else:
            pass

    while len(bbox_list_thresholded) > 0:
        current_box = bbox_list_thresholded.pop(0)
        bbox_list_new.append(current_box)
        for box in bbox_list_thresholded:
            if current_box[4] == box[4]:
                iou = IOU (current_box[:4], box[:4])
                if iou > iou_threshold:
                    bbox_list_thresholded. renove (box)

    return bbox_list_new

#out_np = out[0]["boxes"]
#out_np = out_np.detach().numpy().astype(int)

nms(out[0]["boxes"].detach().numpy())
out_np = bbox_list_new.numpy().astype(int)

def segment():
    image = cv2.imread(r"C:\Users\alina\Downloads\\presidents.jpg", cv2.IMREAD_COLOR)
    for j in range(len(out_np)):
        img = cv2.rectangle(image,(out_np[j][0],out_np[j][1]),(out_np[j][2],out_np[j][3]),(0,255,0),1)
        os.chdir("\\Users\\alina\\jupiter\\Wider_face_dataset_2\\df_test\\")
        cv2.imwrite("presidents_test_mns_22.jpg", img)
            
segment()

IndexError: index 5 is out of bounds for axis 0 with size 4

In [32]:
image = cv2.imread(r"C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\wider_face_train\\0.jpg", cv2.IMREAD_COLOR) 
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) 
image = image/255 
image = torchvision.transforms.ToTensor()(image) 
out = model([image]) 
out_np1 = out[0]["boxes"]
out_np2 = out[0]["scores"]
out_np1 = out_np1.detach().numpy().astype(float).tolist()
out_np2 = out_np2.detach().numpy().astype(float).tolist()
for i in range(len(out_np2)):
    out_np1[i].append( out_np2[i])

In [43]:
def segment(): 
    image =cv2.imread("C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\wider_face_train\\0.jpg", cv2.IMREAD_COLOR) 
    for j in range(len(out_np)): 
        x1=int(out_np1[j][0])
        y1=int(out_np1[j][1])
        x2=int(out_np1[j][2])
        y2=int(out_np1[j][3])
        img = cv2.rectangle(image,(x1,y1),(x2,y2),(0,255,0),1) 
        os.chdir("C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\df_test") 
        cv2.imwrite("test_1_nms__60.jpg", image) 
             
segment()

In [33]:
def intersect(box_a, box_b):
    x1, y1, x2, y2 = box_a
    x3, y3, x4, y4 = box_b
    
    left = max(x1, x3)
    top = min(y2, y4)
    right = min(x2, x4)
    bottom = max(y1, y3)

    width = right - left
    height = top - bottom
    
    if(width<0 or height<0):
        return 0
    else:
        return width*height

In [34]:
def IoU(box_a, box_b):
    inter = intersect(box_a, box_b)
    area_a = (box_a[2]-box_a[0])*(box_a[3]-box_a[1])
    area_b = (box_b[2]-box_b[0])*(box_b[3]-box_b[1])
    return (inter)/(area_a+area_b-inter)

In [37]:
def nms (boxes, conf_threshold=0.06, iou_threshold=0.4): 
    bbox_list_thresholded = []
    bbox_list_new = []
    #Stage 1: (Sort boxes, and filter out boxes with low confidence)
    boxes_sorted=sorted(boxes, reverse=True, key=lambda x : x[4])
    for box in boxes_sorted:
        if box[4] > conf_threshold: 
            bbox_list_thresholded.append(box)
        else:
            pass
    #Stage 2: (Loop over all boxes, and remove boxes with high IOU)
    while len(bbox_list_thresholded)>0: 
        current_box = bbox_list_thresholded.pop(0) 
        bbox_list_new.append(current_box) 
        for box in bbox_list_thresholded: 
            iou=IoU(current_box[:4], box[:4])
            if iou > iou_threshold: 
                bbox_list_thresholded.remove(box)
    return bbox_list_new

In [38]:
ar=nms(out_np1)
p=cv2.imread('C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\wider_face_train\\0.jpg',1)
for j in range(len(ar)):
     cv2.rectangle(p,(int(ar[j][0]),int(ar[j][1])),(int(ar[j][2]),int(ar[j][3])),(0,255,255),1)
os.chdir("C:\\Users\\alina\\jupiter\\Wider_face_dataset_2\\df_test") 
cv2.imwrite("test_3_nms__60.jpg", p) 

True

In [ ]:
print ("hello world")